In [1]:
import tensorflow as tf
import numpy as np
import random
import datetime
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import scipy.sparse as sp
import importlib
import GNN
importlib.reload(GNN)

<module 'GNN' from 'C:\\Users\\BBD\\Desktop\\图计算\\图神经网络\\GNN.py'>

In [2]:
A, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = GNN.load_data_v1('cora')
features = features.toarray()

Dataset has 2708 nodes, 5429 edges, 1433 features.


### 模型存储演示

In [3]:
model = GNN.GNN(layers=[32,32],batch_size=1024,epoch=150,learning_rate=0.05,dropout_rate=0.5,l2_reg=1e-4)
model.fit(features,y_train,train_mask,GNN.get_A_GCN(A))




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
2019-12/03/19 13:48:15	epoch 0	18.738691
2019-12/03/19 13:48:15	epoch 1	10.25875
2019-12/03/19 13:48:15	epoch 2	7.956712
2019-12/03/19 13:48:15	epoch 3	8.285624
2019-12/03/19 13:48:15	epoch 4	7.937876
2019-12/03/19 13:48:16	epoch 5	6.7684326
2019-12/03/19 13:48:16	epoch 6	5.3304586
2019-12/03/19 13:48:16	epoch 7	4.314229
2019-12/03/19 13:48:16	epoch 8	3.8182118
2019-12/03/19 13:48:16	epoch 9	3.6371217
2019-12/03/19 13:48:17	epoch 10	3.5961177
2019-12/03/19 13:48:17	epoch 11	3.5957708
2019-12/03/19 13:48:17	epoch 12	3.6092763
2019-12/03/19 13:48:17	epoch 13	3.63414
2019-12/03/19 13:48:18	epoch 14	3.6527014
2019-12/03/19 13:48:18	epoch 15	3.656833
2019-12/03/19 13:48:18	epoch 16	3.647149
2019-12/03/19 13:48:18	epoch 17	3.635092
2019-12/03/19 13:48:18	epoch 18	3.6265163
2019-12/03/19 13:48:19	epoch 19	3.619433
2019-12/03/19 13:48:19	epoch 20	3.613562
2019-12/03/19 13:48:19	epoch 21	3.6078904
2

In [4]:
model.save_parameters('./model/demo_model')

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

./model/demo_model


In [5]:
model2 = GNN.GNN()
model2.load_parameters('./model/demo_model')

INFO:tensorflow:Restoring parameters from ./model/demo_model


In [6]:
y_train_predict = model2.predict_proba(features,GNN.get_A_GCN(A))
roc_auc_score(y_train[train_mask], y_train_predict[train_mask],average='micro')

0.9998979591836735

### 拟合GCN

In [7]:
model = GNN.GNN(layers=[32,32],batch_size=1024,epoch=150,learning_rate=0.05,dropout_rate=0.5,l2_reg=1e-4)
model.fit(features,y_train,train_mask,GNN.get_A_GCN(A))

2019-12/03/19 13:48:50	epoch 0	21.6821
2019-12/03/19 13:48:50	epoch 1	15.269801
2019-12/03/19 13:48:50	epoch 2	9.7899065
2019-12/03/19 13:48:51	epoch 3	7.631993
2019-12/03/19 13:48:51	epoch 4	6.4846463
2019-12/03/19 13:48:51	epoch 5	5.7786818
2019-12/03/19 13:48:51	epoch 6	5.1373324
2019-12/03/19 13:48:52	epoch 7	4.703798
2019-12/03/19 13:48:52	epoch 8	4.414856
2019-12/03/19 13:48:52	epoch 9	4.2316275
2019-12/03/19 13:48:52	epoch 10	4.1289577
2019-12/03/19 13:48:53	epoch 11	4.0552135
2019-12/03/19 13:48:53	epoch 12	4.001187
2019-12/03/19 13:48:53	epoch 13	3.9554763
2019-12/03/19 13:48:53	epoch 14	3.9367785
2019-12/03/19 13:48:54	epoch 15	3.9328134
2019-12/03/19 13:48:54	epoch 16	3.9318676
2019-12/03/19 13:48:54	epoch 17	3.9248319
2019-12/03/19 13:48:54	epoch 18	3.9190578
2019-12/03/19 13:48:54	epoch 19	3.9148068
2019-12/03/19 13:48:55	epoch 20	3.9094064
2019-12/03/19 13:48:55	epoch 21	3.9030013
2019-12/03/19 13:48:55	epoch 22	3.8940744
2019-12/03/19 13:48:55	epoch 23	3.8841715
2019-12/

训练精度

In [8]:
y_train_predict = model.predict_proba(features,GNN.get_A_GCN(A))
roc_auc_score(y_train[train_mask], y_train_predict[train_mask],average='micro')

0.9999319727891156

Validation精度

In [9]:
y_val_predict = model.predict_proba(features,GNN.get_A_GCN(A))
roc_auc_score(y_val[val_mask], y_val_predict[val_mask],average='micro')

0.9445106666666666

测试精度

In [10]:
y_test_predict = model.predict_proba(features,GNN.get_A_GCN(A))
roc_auc_score(y_test[test_mask], y_test_predict[test_mask],average='micro')

0.9278956666666668

### 拟合普通神经网络

In [11]:
model = GNN.GNN(layers=[32,32],batch_size=1024,epoch=100,learning_rate=0.05,dropout_rate=0.5,l2_reg=1e-4)
model.fit(features,y_train,train_mask)

2019-12/03/19 13:49:24	epoch 0	73.86207
2019-12/03/19 13:49:24	epoch 1	53.267803
2019-12/03/19 13:49:24	epoch 2	41.66118
2019-12/03/19 13:49:24	epoch 3	33.799862
2019-12/03/19 13:49:25	epoch 4	27.939812
2019-12/03/19 13:49:25	epoch 5	23.389029
2019-12/03/19 13:49:25	epoch 6	19.39318
2019-12/03/19 13:49:25	epoch 7	16.178947
2019-12/03/19 13:49:25	epoch 8	13.613396
2019-12/03/19 13:49:26	epoch 9	11.507382
2019-12/03/19 13:49:26	epoch 10	9.718004
2019-12/03/19 13:49:26	epoch 11	8.380995
2019-12/03/19 13:49:26	epoch 12	7.411378
2019-12/03/19 13:49:26	epoch 13	6.654064
2019-12/03/19 13:49:27	epoch 14	6.0119863
2019-12/03/19 13:49:27	epoch 15	5.4625654
2019-12/03/19 13:49:27	epoch 16	5.0282736
2019-12/03/19 13:49:27	epoch 17	4.6625695
2019-12/03/19 13:49:28	epoch 18	4.3710403
2019-12/03/19 13:49:28	epoch 19	4.132715
2019-12/03/19 13:49:28	epoch 20	3.980305
2019-12/03/19 13:49:28	epoch 21	3.8774664
2019-12/03/19 13:49:28	epoch 22	3.8194208
2019-12/03/19 13:49:28	epoch 23	3.7887535
2019-12/03/

训练精度

In [12]:
y_train_predict = model.predict_proba(features)
roc_auc_score(y_train[train_mask], y_train_predict[train_mask],average='micro')

0.5377210884353741

Validation精度

In [13]:
y_val_predict = model.predict_proba(features)
roc_auc_score(y_val[val_mask], y_val_predict[val_mask],average='micro')

0.4427173333333334

测试精度

In [14]:
y_test_predict = model.predict_proba(features)
roc_auc_score(y_test[test_mask], y_test_predict[test_mask],average='micro')

0.46400533333333327